In [1]:
import random
import uuid
from datetime import timedelta, datetime
from faker import Faker
import json
import pandas as pd





In [2]:
# Step 1: Read the Excel file
df = pd.read_excel("real_path_data.xlsx", header=None)

# Step 2: Process the DataFrame
nested_list = [] # The final data nestes list that contain [image_main_name, [path1,path2,path3,...]]
current_product = None # temp variable for storing the current element
images = [] # temp variable for storing the current element
"""
The idea is i have 2 table, products and product_thumbnails and i have to insert data for every product and their thumbnails also,
about the thumbnails data we can have multiple thumbnails for a product(refer throught foreign key in database). But in our project,
because limitation of storage so the devs decide to use a thumbnail set for multiple product. This code section is to get the thumbnails
data from the xlsx file.
"""
for index, row in df.iterrows():
    product_name = row[0]
    image_name = row[1]
    """
    Comment tiếng việt =))
    Vòng for duyệt qua tất cả các row của dataset, lấy ra product_name và image_name tại dòng đó, kiểm tra nếu không phải NULL thì gán cho current_product và image_name,
    duyệt tới khi nào gặp 1 product mới thì append cặp trước đó vào nested_list.
    Lưu ý: Với merged cells, pandas chỉ coi cell gốc là có giá trị, còn lại thì đều là NaN
    """
    if pd.notna(product_name): #notna is to check notnull
        if current_product:
            nested_list.append([current_product, images])
        current_product = product_name
        images = [image_name] if pd.notna(image_name) else []
    else:
        if pd.notna(image_name):
            images.append(image_name)
if current_product:
    nested_list.append([current_product, images])

# Step 3: Print the nested list (this list is data for image name and image path)
print(nested_list)

[['Blooming Joy', ['https://res.cloudinary.com/dl0g9euyy/image/upload/v1729788573/thumbnail_1_tnu9z0.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729788573/thumbnail_2_pony5d.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729788573/Blooming_Joy_donorf.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729788573/thumbnail_3_qduql9.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729788573/thumbnail_4_ewq4su.jpg']], ['Bohemian Pink', ['https://res.cloudinary.com/dl0g9euyy/image/upload/v1729789258/Bohemian_Pink_oj4bom.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729789258/thumbnail_1_h6ujzm.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729789259/thumbnail_2_ltov2r.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729789261/thumbnail_3_qutlvn.jpg', 'https://res.cloudinary.com/dl0g9euyy/image/upload/v1729789262/thumbnail_4_crmv2d.jpg']], ['Cactus In Orange', ['https://res.cloudinary.com/dl0g9euyy/image/upload/v1

In [4]:
# Initialize Faker for generating realistic timestamps
fake = Faker()

# Constants
CREATOR_ID = "6757a1bd-4112-4710-a317-ad86cd0c82c7"

# Load JSON and nested image list
with open('../data/data_cay.json') as f: # Load products data that crawled from mowgarden
    products = json.load(f)

# Nested list above
nested_images = nested_list

# Generate a random 20-character string for product code
def generate_random_string(length=20):
    return ''.join(random.choices('abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ0123456789', k=length))

# Generate random timestamps within a range
def random_timestamp():
    return fake.date_time_between(start_date="-2y", end_date="now")

# Write SQL insert statements to file
with open("insert_statements_new.sql", "w") as file:
    for product in products:
        # Generate data for each product
        product_id = str(uuid.uuid4())
        product_code = generate_random_string()
        summary = " ".join(product['summary']) if isinstance(product['summary'], list) else product['summary']
        content = " ".join(product['content']) if isinstance(product['content'], list) else product['content']
        status = random.choice([True, False])
        price = product['price']
        # Generate realistic timestamps within an acceptable range
        created_at = random_timestamp()

        # Calculate max_days and ensure it's valid for `updated_at`
        max_days = (datetime.now() - created_at).days
        if max_days > 0:
            updated_at = created_at + timedelta(days=random.randint(1, max_days))
        else:
            updated_at = created_at  # No days to add, so `updated_at` is the same as `created_at`

        deleted_at = "NULL"
        stock = random.randint(1, 100)
        sold = random.randint(1, 100)
        product_name = product['name']

        # Select an image set and main image for the product
        image_set = random.choice(nested_images)
        main_image_path = image_set[1][0]
        main_image_name = main_image_path.split("/")[-1]

        # Insert statement for the product
        product_insert = f"""
        INSERT INTO products (id, product_code, summary, content, status, creator_id, price,
                             created_at, updated_at, deleted_at, stock, sold, product_name, image_path, image_name)
        VALUES ('{product_id}', '{product_code}', '{summary}', '{content}', {status}, '{CREATOR_ID}', 
                {price}, '{created_at}', '{updated_at}', {deleted_at}, {stock}, {sold}, '{product_name}', 
                '{main_image_path}', '{main_image_name}');
        """
        file.write(product_insert + "\n")

        # Insert statements for product thumbnails
        if isinstance(image_set[1], list):
            for image_url in image_set[1]:
                thumbnail_id = str(uuid.uuid4())
                thumbnail_created_at = random_timestamp()
                max_days = (datetime.now() - thumbnail_created_at).days
                if max_days > 0:
                    thumbnail_updated_at = thumbnail_created_at + timedelta(days=random.randint(1, max_days))
                else:
                    thumbnail_updated_at = thumbnail_created_at  # No days to add, so `updated_at` is the same as `created_at`
                thumbnail_insert = f"""
                INSERT INTO product_thumbnails (id, product_id, image_path, image_name,
                                                created_at, updated_at, deleted_at)
                VALUES ('{thumbnail_id}', '{product_id}', '{image_url}', '{image_url.split('/')[-1]}', 
                        '{thumbnail_created_at}', '{thumbnail_updated_at}', NULL);
                """
                file.write(thumbnail_insert + "\n")

print("SQL insert statements have been written to 'insert_statements_new.sql'")
# This .sql file can be give to the dev for data avalibility


SQL insert statements have been written to 'insert_statements_new.sql'
